In [3]:
def calculateDims(coordinates:list, x_shape:int, y_shape:int): 
    #Remember that the coordinates are written like x1;y1;x2;y2;x3;y3
    if type(coordinates)!=list:
        print(type(coordinates))
        raise ValueError
    centerWidthHeights = [] #The format will be [[center0,width0,height0],[center1,width1,height1],.........]
    
    
    for coords in coordinates:
        xs = []
        ys = []
        cenwidhgt = [] #These are the [centerX,widthX,heightX] described above
                       #centerX will be written as xX,yX => [centerX,widthX,heightX]  = [xX,yX,widthX,heightX]

        coordslist = coords.split(";") #Remember that every value indexed at a even num is xs, and odd num is ys
        for i in range(len(coordslist)):
            if i%2==0: 
                xs.append(int(coordslist[i]))
            else:
                ys.append(int(coordslist[i]))
        xsMax = max(xs)
        xsMin = min(xs)
        ysMax = max(ys)
        ysMin = min(ys)     
        xsCen = ((xsMin+xsMax)/2)/x_shape
        ysCen = ((ysMin+ysMax)/2)/y_shape
        height = abs(ysMax-ysMin)/y_shape #should be positive, but just to be sure
        width = abs(xsMax-xsMin)/x_shape
        if width == 0 and height == 0: 
            height = 0 
            width = 0
        cenwidhgt = [xsCen,ysCen,width,height]
        centerWidthHeights.append(cenwidhgt)
    
    return centerWidthHeights

In [4]:
#Read in the textfile with wher the nodule imformation is pou up like this: 
    # NoduleID: SOPInstanceUID: png path: coordinates: 
    # Ex:
    # Nodule 01,1.3.2.4.6.3.12413......,/home/heris/............../image1.png,x1;y1;x2;y2;x3;y3

def boundingBoxDim(input_txt_path:str, output_txt_path:str):
    try:
        NoduleIDs = []
        CoordinatesTot = []
        with open(input_txt_path, "r") as txt:
            lines = txt.readlines()
            for line in lines:
                #Each line is in a csv format so can split on ","
                line_split = line.split(",")
                noduleID, SOPInstanceUID, img_path = line_split[0], line_split[1], line_split[2]
                NoduleIDs.append(noduleID)   
                x_shape, y_shape = int(line_split[3]), int(line_split[4])
                coordinates = line_split[5]
                CoordinatesTot.append(coordinates) 
        centersWidthsHeigths = calculateDims(CoordinatesTot,x_shape, y_shape)
        ##################################################################
        #if the lesion is a point lesion, the txt file is then put into tmp
        #Point lesions will be the top point of the tumor ad non-nodules.
        output_txt_copy_path = output_txt_path
        output_tmp_path = "    " #tmp path
        output_txt_path = output_tmp_path
        for dims in centersWidthsHeigths: 
            if dims[-1] != 0 and dims[-2] != 0:
                output_txt_path = output_txt_copy_path
        ##################################################################

    except FileNotFoundError:
        print(f"File {input_txt_path} not found.")
    except SyntaxError:
        print(f"Wrong imput syntax!")
    
    #JASON or TXT, read up on YOLO v8
    try:
        with open(output_txt_path, 'w') as file:
            # Write the content to the file
            
            #Lesion id = 0
            for dims in centersWidthsHeigths:
                #######
                #Modify the following line, take into accunt spherecy and malignincy to make other lesion labels etc lesion with malign < 3 & spherecy < 3 => Label 0, malign < 3 => Label 1, spherecy < 3 => Label 2, malign > 3 & spherecy > 3 => Label 3 etc. 
                file.write("0 ") #Lesion ID = 0
                ####### 
                for i in range(len(dims)):
                    if i != len(dims)-1:
                        file.write(f"{dims[i]} ")
                    else: 
                        file.write(f"{dims[i]}\n")
               
            
        print(f"Text has been written to {output_txt_path}.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return SOPInstanceUID, img_path, NoduleIDs, centersWidthsHeigths 

In [5]:
import os
import shutil
import math 

In [16]:
#Automated script 
output_folder_path = "/home/heris/data/RAW/YOLO_dataset"
input_txt_path = "/home/heris/data/RAW/LIDC-IDRI-dicom/"
if not os.path.exists(output_folder_path):
    os.mkdir(output_folder_path)

train_val_test = ["/train","/validation","/test"]
for elem in train_val_test:
    if not os.path.exists(output_folder_path+elem):
        os.mkdir(output_folder_path+elem)

n_patients, n_train, n_val, n_test = 0, 0, 0, 0
i = 0
for root, dirs, files in os.walk(input_txt_path):
    if len(dirs) == 0:
        for file in files: 
            if file[-1] == "t":
                if not os.path.exists(output_folder_path+"/"+file):
                    if os.path.exists(root+"/"+file[0:-3]+"png"):
                        boundingBoxDim(root+"/"+file,output_folder_path+"/"+file)
                        shutil.copy(root+"/"+file[0:-3]+"png",output_folder_path)
    
    
    


Text has been written to /home/heris/data/RAW/YOLO_dataset/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.266692499875116030661036210807.txt.
Text has been written to /home/heris/data/RAW/YOLO_dataset/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.222098252047357192090439228841.txt.
Text has been written to /home/heris/data/RAW/YOLO_dataset/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.286608731982968057118253662928.txt.
Text has been written to /home/heris/data/RAW/YOLO_dataset/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.139636132253744151113715840194.txt.
Text has been written to /home/heris/data/RAW/YOLO_dataset/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.629488498434344434501396689350.txt.
Text has been written to /home/heris/data/RAW/YOLO_dataset/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.285915778169973868115460699672.txt.
Text has been written to /home/heris/data/RAW/YOLO_dataset/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.225900589792147134785051710110.txt.
Text has been written to /home/heris/data/RAW/YOLO_data